# 🚀 Basic Federated Learning Experiment

This notebook will guide you through running your first federated learning experiment using our framework. You'll learn how to:

- Set up a basic FL experiment
- Configure clients and server
- Run training with different strategies
- Monitor and analyze results

## 📋 Prerequisites

Make sure you've completed the introduction notebook and have all dependencies installed.

In [ ]:
import sys
import os
import subprocess
import time
import threading
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add framework to path
framework_root = Path().absolute().parent
sys.path.insert(0, str(framework_root))

print(f"Framework root: {framework_root}")
print(f"Current working directory: {Path.cwd()}")

## 🔧 Experiment Configuration

Let's start by configuring our first federated learning experiment:

In [ ]:
# Experiment configuration
config = {
    'num_clients': 5,           # Number of federated clients
    'num_rounds': 10,           # Number of federation rounds
    'dataset': 'MNIST',         # Dataset to use (MNIST, FMNIST, CIFAR10)
    'strategy': 'fedavg',       # Aggregation strategy
    'attack': 'none'            # No attacks for this basic experiment
}

print("🎯 Experiment Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 🏃‍♂️ Method 1: Using the Automated Script

The easiest way to run an experiment is using our automated script:

In [ ]:
def run_basic_experiment(config):
    """Run a basic federated learning experiment using the automated script."""
    
    # Construct the command
    script_path = framework_root / "experiment_runners" / "run_with_attacks.py"
    
    cmd = [
        sys.executable,
        str(script_path),
        "--num-clients", str(config['num_clients']),
        "--rounds", str(config['num_rounds']),
        "--dataset", config['dataset'],
        "--strategy", config['strategy'],
        "--attack", config['attack']
    ]
    
    print(f"🚀 Running command: {' '.join(cmd)}")
    print("\n" + "="*50)
    print("Starting Federated Learning Experiment...")
    print("="*50)
    
    # Run the experiment
    try:
        # Change to the framework root directory
        original_cwd = os.getcwd()
        os.chdir(framework_root)
        
        # Run the experiment
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        # Restore original directory
        os.chdir(original_cwd)
        
        print("\n📊 Experiment Output:")
        print("-" * 30)
        if result.stdout:
            print(result.stdout)
        if result.stderr:
            print("Errors/Warnings:")
            print(result.stderr)
            
        print(f"\n✅ Experiment completed with return code: {result.returncode}")
        return result.returncode == 0
        
    except subprocess.TimeoutExpired:
        print("⏰ Experiment timed out after 5 minutes")
        return False
    except Exception as e:
        print(f"❌ Error running experiment: {e}")
        return False

# Run the experiment
success = run_basic_experiment(config)
print(f"\n🎯 Experiment {'succeeded' if success else 'failed'}!")

## 🛠️ Method 2: Manual Server and Client Setup

For better understanding, let's also see how to run server and clients manually:

In [ ]:
def start_server_process(config):
    """Start the federated learning server in a separate process."""
    server_path = framework_root / "core" / "server.py"
    
    cmd = [
        sys.executable,
        str(server_path),
        "--rounds", str(config['num_rounds']),
        "--dataset", config['dataset'],
        "--strategy", config['strategy']
    ]
    
    print(f"🖥️ Starting server with command: {' '.join(cmd)}")
    
    # Change to framework directory and start server
    original_cwd = os.getcwd()
    os.chdir(framework_root)
    
    server_process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    os.chdir(original_cwd)
    return server_process

def start_client_process(client_id, config):
    """Start a federated learning client."""
    client_path = framework_root / "core" / "client.py"
    
    cmd = [
        sys.executable,
        str(client_path),
        "--cid", str(client_id),
        "--dataset", config['dataset']
    ]
    
    print(f"👤 Starting client {client_id} with command: {' '.join(cmd)}")
    
    # Change to framework directory and start client
    original_cwd = os.getcwd()
    os.chdir(framework_root)
    
    client_process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    os.chdir(original_cwd)
    return client_process

print("🔧 Manual setup functions defined. Ready to run manual experiment if needed.")

## 📊 Understanding the Experiment Flow

Let's break down what happens during a federated learning experiment:

In [ ]:
def explain_fl_process():
    """Explain the federated learning process step by step."""
    
    print("🔄 Federated Learning Process Overview:\n")
    
    steps = [
        "1. Server Initialization",
        "   • Load initial model parameters",
        "   • Initialize aggregation strategy",
        "   • Start listening for client connections",
        "",
        "2. Client Connection",
        "   • Clients connect to server",
        "   • Load local datasets",
        "   • Initialize local models",
        "",
        "3. Federation Round (repeated)",
        "   • Server sends global model to selected clients",
        "   • Clients perform local training",
        "   • Clients send updates back to server",
        "   • Server aggregates updates using chosen strategy",
        "   • Server evaluates global model",
        "",
        "4. Completion",
        "   • Final model evaluation",
        "   • Results logging and cleanup"
    ]
    
    for step in steps:
        print(step)
    
    print("\n" + "="*50)
    print("Key Concepts:")
    print("• Local Training: Each client trains on its own data")
    print("• Model Updates: Clients send parameter updates, not raw data")
    print("• Aggregation: Server combines updates using chosen strategy")
    print("• Privacy: Raw data never leaves the client")

explain_fl_process()

## 🔍 Monitoring and Analysis

Let's check what results were generated and how to analyze them:

In [ ]:
def check_experiment_results():
    """Check for experiment results and logs."""
    
    # Look for log files
    log_files = []
    for log_pattern in ['*.log', 'experiment_results/*', 'enhanced_experiment_results/*']:
        log_files.extend(list(framework_root.glob(log_pattern)))
    
    print("📁 Found experiment files:")
    if log_files:
        for log_file in log_files[:10]:  # Show first 10 files
            file_size = log_file.stat().st_size if log_file.exists() else 0
            print(f"  • {log_file.name} ({file_size} bytes)")
        if len(log_files) > 10:
            print(f"  ... and {len(log_files) - 10} more files")
    else:
        print("  No experiment files found yet.")
    
    # Check for recent log entries
    recent_logs = []
    for log_file in framework_root.glob('*.log'):
        if log_file.stat().st_size > 0:
            recent_logs.append(log_file)
    
    if recent_logs:
        print(f"\n📋 Recent activity in {len(recent_logs)} log files:")
        for log_file in recent_logs[:3]:  # Show first 3 log files
            try:
                with open(log_file, 'r') as f:
                    lines = f.readlines()
                    if lines:
                        print(f"\n{log_file.name} (last few lines):")
                        for line in lines[-3:]:
                            print(f"  {line.strip()}")
            except Exception as e:
                print(f"  Could not read {log_file.name}: {e}")

check_experiment_results()

## 🎯 Comparing Different Strategies

Let's run quick experiments with different aggregation strategies to see their behavior:

In [ ]:
def run_strategy_comparison():
    """Run quick experiments with different strategies for comparison."""
    
    strategies = ['fedavg', 'fedprox', 'fedavgm']
    results = {}
    
    print("🔄 Running strategy comparison experiments...")
    print("(Using smaller configuration for faster execution)\n")
    
    quick_config = {
        'num_clients': 3,
        'num_rounds': 5,
        'dataset': 'MNIST',
        'attack': 'none'
    }
    
    for strategy in strategies:
        print(f"📊 Testing {strategy.upper()}...")
        
        config = quick_config.copy()
        config['strategy'] = strategy
        
        # Run experiment (with shorter timeout)
        start_time = time.time()
        success = run_basic_experiment(config)
        end_time = time.time()
        
        results[strategy] = {
            'success': success,
            'duration': end_time - start_time
        }
        
        print(f"  Result: {'✅ Success' if success else '❌ Failed'} ({results[strategy]['duration']:.1f}s)\n")
    
    # Summary
    print("📈 Strategy Comparison Summary:")
    print("-" * 40)
    for strategy, result in results.items():
        status = "✅" if result['success'] else "❌"
        print(f"{strategy:10} {status} ({result['duration']:.1f}s)")
    
    return results

# Uncomment the line below to run the comparison (may take several minutes)
# comparison_results = run_strategy_comparison()
print("💡 Tip: Uncomment the line above to run a strategy comparison experiment")

## 📈 Simulated Results Visualization

Let's create some example visualizations to show what typical FL results look like:

In [ ]:
def create_example_visualizations():
    """Create example visualizations of typical FL experiment results."""
    
    # Simulate typical FL training curves
    rounds = np.arange(1, 11)
    
    # Simulate different strategy behaviors
    strategies = {
        'FedAvg': {
            'accuracy': [0.1, 0.45, 0.62, 0.71, 0.77, 0.81, 0.84, 0.86, 0.87, 0.88],
            'loss': [2.3, 1.8, 1.4, 1.1, 0.9, 0.75, 0.65, 0.58, 0.53, 0.50]
        },
        'FedProx': {
            'accuracy': [0.1, 0.42, 0.65, 0.74, 0.79, 0.83, 0.85, 0.87, 0.88, 0.89],
            'loss': [2.3, 1.9, 1.3, 1.0, 0.85, 0.70, 0.62, 0.55, 0.51, 0.48]
        },
        'FedAvgM': {
            'accuracy': [0.1, 0.48, 0.68, 0.76, 0.81, 0.84, 0.86, 0.88, 0.89, 0.90],
            'loss': [2.3, 1.7, 1.2, 0.95, 0.78, 0.68, 0.60, 0.54, 0.49, 0.46]
        }
    }
    
    # Create plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Accuracy plot
    ax1.set_title('📊 Federated Learning Accuracy Comparison', fontsize=14, fontweight='bold')
    for strategy, metrics in strategies.items():
        ax1.plot(rounds, metrics['accuracy'], marker='o', linewidth=2, label=strategy)
    
    ax1.set_xlabel('Federation Round')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)
    
    # Loss plot
    ax2.set_title('📉 Federated Learning Loss Comparison', fontsize=14, fontweight='bold')
    for strategy, metrics in strategies.items():
        ax2.plot(rounds, metrics['loss'], marker='s', linewidth=2, label=strategy)
    
    ax2.set_xlabel('Federation Round')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Create a summary table
    summary_data = []
    for strategy, metrics in strategies.items():
        summary_data.append({
            'Strategy': strategy,
            'Final Accuracy': f"{metrics['accuracy'][-1]:.3f}",
            'Final Loss': f"{metrics['loss'][-1]:.3f}",
            'Accuracy Improvement': f"{metrics['accuracy'][-1] - metrics['accuracy'][0]:.3f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print("\n📋 Example Results Summary:")
    print(summary_df.to_string(index=False))
    
    print("\n💡 Key Observations:")
    print("  • FedAvgM shows fastest convergence due to server-side momentum")
    print("  • FedProx provides stable training with regularization")
    print("  • FedAvg serves as a reliable baseline")
    print("  • All strategies show typical FL convergence patterns")

create_example_visualizations()

## 🎓 Key Takeaways

From this notebook, you've learned:

### ✅ What You Accomplished
1. **Set up and ran** your first federated learning experiment
2. **Understood** the FL process flow and key concepts
3. **Explored** different ways to run experiments (automated vs manual)
4. **Learned** how to monitor and analyze results
5. **Compared** different aggregation strategies

### 🔍 Understanding Federated Learning
- **Data Privacy**: Raw data never leaves client devices
- **Distributed Training**: Each client trains on local data
- **Model Aggregation**: Server combines updates without seeing data
- **Iterative Process**: Multiple rounds improve global model

### 🛠️ Technical Skills
- Running FL experiments with our framework
- Configuring different strategies and parameters
- Monitoring experiment progress and results
- Basic result visualization and analysis

## 🚀 Next Steps

Ready to dive deeper? Here's what to explore next:

1. **[03_Aggregation_Strategies_Deep_Dive.ipynb](./03_Aggregation_Strategies_Deep_Dive.ipynb)** - Detailed exploration of each strategy
2. **[04_Attack_Scenarios_and_Security.ipynb](./04_Attack_Scenarios_and_Security.ipynb)** - Security testing with various attacks
3. **[05_Comprehensive_Experiments_with_run_extensive_experiments.ipynb](./05_Comprehensive_Experiments_with_run_extensive_experiments.ipynb)** - Large-scale automated experiments

## 🔧 Troubleshooting Tips

If you encountered issues:

- **Port conflicts**: Make sure ports 8080 is available
- **Dependencies**: Ensure all required packages are installed
- **Paths**: Verify the framework root path is correct
- **Logs**: Check log files for detailed error messages
- **Timeout**: Increase timeout values for slower systems

**Happy Federated Learning! 🎉**